In [ ]:
{-# LANGUAGE ScopedTypeVariables #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE RankNTypes, BangPatterns #-}
import Control.Monad
import Control.Monad.Primitive

import qualified Data.Vector.Generic as G
import qualified Data.Vector.Unboxed as U

import Numeric.SpecFunctions
import Numeric.MathFunctions.Constants 
import Numeric.MathFunctions.Comparison

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy

import Debug.Trace

:l NB/Plot

In [ ]:
igammaRoundtripError :: Double -> Double -> Double
igammaRoundtripError z p
  = m_epsilon/2 * (1 + abs (x * f' / p))
  where
    x  = invIncompleteGamma z p
    f' = exp ( log x * (z-1) - x - logGamma z)

plotErrorEstimate z
  = toRenderable
  $ layout_title .~ "Error estimate"
  $ plotFunctions [ \p -> logBase 10 $ relativeError p (incompleteGamma z $ invIncompleteGamma z p)
  --relativeError p (roudtripIGamma z p)
                  , logBase 10 . igammaRoundtripError z
                  , logBase 10 . const (m_epsilon/2)
                  ] (0,1)

In [ ]:
plotErrorEstimate 200

In [ ]:
toRenderable $
  plotFunctions [ invIncompleteGamma 10] (0,1)
toRenderable $
  plotFunctions [ incompleteGamma (2.1e5)] (0,4e6)

-- plotErrorEstimate 300


In [ ]:

toRenderable
  $ layout_title .~ "Error estimate"
  $ plotFunctions [ logBase 10 . igammaRoundtripError 10
                  ] (0,1)

In [ ]:
incompleteGamma 1000 3000

In [ ]:
incompleteGamma' :: Double       -- ^ /z/ ∈ (0,∞)
                -> Double       -- ^ /x/ ∈ (0,∞)
                -> Double
incompleteGamma' p x
    | isNaN p || isNaN x = m_NaN
    | x < 0 || p <= 0    = m_pos_inf
    | x == 0             = 0
    -- For very large `p' normal approximation gives <1e-10 error
    | p >= 2e5           = norm (3 * sqrt p * ((x/p) ** (1/3) + 1/(9*p) - 1))
    | p >= 500           = approx
    -- Dubious approximation
    | x >= 1e8           = 1
    | x <= 1 || x < p    = let a = p * log x - x - logGamma (p + 1)
                               g = a + log (pearson p 1 1)
                           in if g > limit then exp g else 0
    | otherwise          = let g = p * log x - x - logGamma p + log cf
                           in if g > limit then 1 - exp g else 1
  where
    -- CDF for standard normal distributions
    norm a = 0.5 * erfc (- a / m_sqrt_2)
    -- For large values of `p' we use 18-point Gauss-Legendre
    -- integration.
    approx
--      | traceShow ans False = undefined
      | ans > 0  = 1 - ans
      | ans == 0 && x > p1 = 1 - ans
      | otherwise = -ans
      where
        -- Set upper limit for integration
        xu | x > p1    =         (p1 + 11.5*sqrtP1) `max` (x + 6*sqrtP1)
           | otherwise = max 0 $ (p1 -  7.5*sqrtP1) `min` (x - 5*sqrtP1)
        s = U.sum $ U.zipWith go coefY coefW
        go y w = let t = x + (xu - x)*y
                 in w * exp( -(t-p1) + p1*(log t - lnP1) )
        ans = s * (xu - x) * exp( p1 * (lnP1 - 1) - logGamma p)
        --
        p1     = p - 1
        lnP1   = log  p1
        sqrtP1 = sqrt p1
    --
    pearson !a !c !g
        | c' <= tolerance = g'
        | otherwise       = pearson a' c' g'
        where a' = a + 1
              c' = c * x / a'
              g' = g + c'
    cf = let a = 1 - p
             b = a + x + 1
             p3 = x + 1
             p4 = x * b
         in contFrac a b 0 1 x p3 p4 (p3/p4)
    contFrac !a !b !c !p1 !p2 !p3 !p4 !g
        | abs (g - rn) <= min tolerance (tolerance * rn) = g
        | otherwise = contFrac a' b' c' (f p3) (f p4) (f p5) (f p6) rn
        where a' = a + 1
              b' = b + 2
              c' = c + 1
              an = a' * c'
              p5 = b' * p3 - an * p1
              p6 = b' * p4 - an * p2
              rn = p5 / p6
              f n | abs p5 > overflow = n / overflow
                  | otherwise         = n
    limit     = -88
    tolerance = 1e-14
    overflow  = 1e37


-- Coefficients for 18-point Gauss-Legendre integration. They are
-- used in implementation of incomplete gamma and beta functions.
coefW,coefY :: U.Vector Double
coefW = U.fromList [ 0.0055657196642445571, 0.012915947284065419, 0.020181515297735382
                   , 0.027298621498568734,  0.034213810770299537, 0.040875750923643261
                   , 0.047235083490265582,  0.053244713977759692, 0.058860144245324798
                   , 0.064039797355015485,  0.068745323835736408, 0.072941885005653087
                   , 0.076598410645870640,  0.079687828912071670, 0.082187266704339706
                   , 0.084078218979661945,  0.085346685739338721, 0.085983275670394821
                   ]
coefY = U.fromList [ 0.0021695375159141994, 0.011413521097787704, 0.027972308950302116
                   , 0.051727015600492421,  0.082502225484340941, 0.12007019910960293
                   , 0.16415283300752470,   0.21442376986779355,  0.27051082840644336
                   , 0.33199876341447887,   0.39843234186401943,  0.46931971407375483
                   , 0.54413605556657973,   0.62232745288031077,  0.70331500465597174
                   , 0.78649910768313447,   0.87126389619061517,  0.95698180152629142
                   ]

In [ ]:
toRenderable $
  plotFunctions [ incompleteGamma' 1000] (0,3000)


In [ ]:
12 / exp (logGamma 1000)